In [1]:
import os
import numpy as np

from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
data_dir = "img"

In [3]:
X = [] 
Y = []
for digit in range(10): # => 0 to 9
    digit_dir = os.path.join(data_dir, str(digit)) # eg:- img/0
    for img_file in os.listdir(digit_dir):
        image = Image.open(os.path.join(digit_dir, img_file)) # => opening image file
        image = image.resize((64, 64)) # resizing the image to 64x64 resolution
        
        #Normal image => (R, G, B) 
        #grayscale image => (L) 
        # (0) -> black
        #(1-254) => shades of gray
        # (255) -> white
        image = image.convert("L") # converting image to grayscale
        
        image = np.array(image) / 255.0 #converting to an array => [255, 255, 255, 0, 0, 255] / 255 =>  [1, 1, 1, 0, 0, 1]
        
        X.append(image)
        Y.append(digit)

X = np.array(X)
Y = np.array(Y)

In [4]:
Image.fromarray(X[925]*255).show() # viewing the image instance from dataset
print(Y[925]) #corresponding output or label

9


In [5]:
print(X.shape)
print(Y.shape)

(960, 64, 64)
(960,)


In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [7]:
label_binarizer = LabelBinarizer()

In [8]:
Y_train = label_binarizer.fit_transform(Y_train) #binary format to work with ML models
Y_test = label_binarizer.transform(Y_test)

In [9]:
# CNN Model
model = models.Sequential([ #stack of layer
    
    # convolution layer
    # add filter to the sample image -> extract features
    # matrix multiplication with a loopy pattern/filter -> feature map
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    
    #8x8
    
    #Pooling layer
    #reduce the dimensions of samples to reduce computation
    layers.MaxPooling2D((2, 2)), 
    #(2x2)
    layers.Flatten(), # converting to one dimensional vector array (1,n) (x, y, z, ....)
    
    #fully connected layer
    layers.Dense(128, activation='relu'), # define neutral network for training => 128 neurons
    layers.Dense(10, activation='softmax') #define output neutral network for predicting => 10 neurons => 0-9(10 digits)
])

In [10]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_test, Y_test)) #epoch => cycles; each cycle our models learns whole dataset once

Epoch 1/10
24/24 [==============================] - 4s 131ms/step - loss: 2.3409 - accuracy: 0.3711 - val_loss: 0.9485 - val_accuracy: 0.8542
Epoch 2/10
24/24 [==============================] - 3s 121ms/step - loss: 0.4506 - accuracy: 0.9479 - val_loss: 0.2525 - val_accuracy: 0.9583
Epoch 3/10
24/24 [==============================] - 3s 120ms/step - loss: 0.1126 - accuracy: 0.9883 - val_loss: 0.1311 - val_accuracy: 0.9635
Epoch 4/10
24/24 [==============================] - 3s 119ms/step - loss: 0.0467 - accuracy: 0.9974 - val_loss: 0.0802 - val_accuracy: 0.9948
Epoch 5/10
24/24 [==============================] - 3s 118ms/step - loss: 0.0217 - accuracy: 1.0000 - val_loss: 0.0665 - val_accuracy: 0.9896
Epoch 6/10
24/24 [==============================] - 3s 117ms/step - loss: 0.0136 - accuracy: 1.0000 - val_loss: 0.0574 - val_accuracy: 0.9896
Epoch 7/10
24/24 [==============================] - 3s 119ms/step - loss: 0.0096 - accuracy: 1.0000 - val_loss: 0.0534 - val_accuracy: 0.9844
Epoch 

In [13]:
test_loss, test_acc = model.evaluate(X_test, Y_test)

print(f"Test Loss: {test_loss*100}")
print(f"Test accuracy: {test_acc*100}")

6/6 [==============================] - 0s 20ms/step - loss: 0.0419 - accuracy: 0.9844
Test Loss: 4.18940968811512
Test accuracy: 98.4375


In [30]:
# Load and preprocess the test image
test_image_path = "img/1/65.png"

test_image = Image.open(test_image_path) #opening the test image

test_image.show()

test_image = test_image.resize((64, 64)) #resizing the user-input image to 64x64 resolution
test_image = test_image.convert("L") #converting user-input image to grayscale format             
test_image = np.array(test_image) / 255.0 # current format of (64, 64)

test_image = np.expand_dims(test_image, 0) #(1, 64, 64) 

# Make predictions
predictions = model.predict(test_image) # return array of probability
print(predictions)
predicted_digit = np.argmax(predictions) # return the index which has highest probability


print("Predicted Digit:", predicted_digit)

1/1 [==============================] - 0s 34ms/step
[[5.9138388e-06 9.9934047e-01 7.8126395e-05 3.7168029e-05 6.2264386e-05
  3.4349807e-05 9.9070712e-05 1.3977451e-07 3.4149823e-04 1.0081383e-06]]
Predicted Digit: 1


In [ ]:
# import pickle
# pickle_out = open("model.pkl", "wb")
# pickle.dump(model, pickle_out)
# pickle_out.close()

In [29]:
model.save("App/model")

INFO:tensorflow:Assets written to: App/model\assets


INFO:tensorflow:Assets written to: App/model\assets
